### Install packages

Note that _hazard_ requires many more packages than _physrisk_, reflecting its role in generating indicators. There are a couple of ways to install the necessary packages.
#### 1 Install using pipenv via command line (recommended)

e.g. from Elyra JupyterLab, go File -> New -> Terminal to create a new terminal, then:

    cd hazard
    pipenv install

#### 2 Install using pip in notebook

First it's necessary to create requirements.txt from Pipfile and then install via pip:

    pipenv requirements > requirements.txt
    pip install -r requirements.txt

https://pipenv.pypa.io/en/latest/advanced/  
Might be useful for those who like to install in the notebook itself, but note that this appears to be a recent feature of pipenv (might not be installed in older images).

### Insert bucket name and credentials as environment variables
Need file credentials.env at top level (i.e. same level as LICENSE), containing:

OSC_S3_BUCKET=... \
OSC_S3_ACCESS_KEY=... \
OSC_S3_SECRET_KEY=...

In [ ]:
import os, sys  # noqa: E402

sys.path.append("../src")
import hazard.utilities.zarr_utilities as zarr_utilities  # noqa: E402

os.environ["CREDENTIAL_DOTENV_DIR"] = os.path.dirname(os.getcwd())
zarr_utilities.set_credential_env_variables()

### Run cut-down hazard indicator creation job

In [1]:
import sys
from dask.distributed import Client, LocalCluster  # noqa: E402
import logging  # noqa: E402
from hazard.docs_store import DocStore  # type: ignore # noqa: E402
from hazard.models.days_tas_above import DaysTasAboveIndicator  # noqa: E402
from hazard.sources.nex_gddp_cmip6 import NexGddpCmip6  # noqa: E402
from hazard.sources.ukcp18_rcp85 import Ukcp18Rcp85
from hazard.sources.osc_zarr import OscZarr  # noqa: E402
import zarr

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(filename="batch.log"),
        logging.StreamHandler(sys.stdout),
    ],
)

docs_store = DocStore(prefix="hazard_test")

cluster = LocalCluster(processes=False)
client = Client(cluster)

gcm = "ukcp18"
scenario = "rcp85"
year = 2030
import os
source = Ukcp18Rcp85(os.environ["CEDA_FTP_URL"], os.environ["CEDA_USERNAME"], os.environ["CEDA_FTP_PASSWORD"])
# target = OscZarr(prefix="hazard_test")  # test prefix is "hazard_test"; main one "hazard"

# as an alternative, we can create a local target:
test_output_dir = os.path.join(os.getcwd(), "src/test/test_output")
store = zarr.DirectoryStore(os.path.join(test_output_dir, 'hazard', 'hazard.zarr'))
target = OscZarr(store=store)

# cut down the transform
model = DaysTasAboveIndicator(
    threshold_temps_c=[15],
    window_years=1,
    gcms=[gcm],
    scenarios=[scenario],
    central_years=[year],
)
# to do the full job, simply:
# model = DaysTasAboveIndicator()

# docs_store.update_inventory(model.inventory())

model.run_all(source, target, client=client)
# can also do:
# items = list(model.batch_items())
# model.run_single(items[0], source, target, client=client)

/Users/ciaran/dev/os-climate-hazard/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2024-07-18 10:22:10,932] {multi_year_average.py:112} INFO - Calculating average indicator for batch item gcm=ukcp18, scenario=rcp85, central_year=2030, years=[2030]
[2024-07-18 10:22:14,491] {days_tas_above.py:56} INFO - Starting calculation for year 2030
[2024-07-18 10:22:21,905] {days_tas_above.py:68} INFO - Data used in DaysTasAboveIndicator
[2024-07-18 10:22:21,905] {days_tas_above.py:69} INFO - <xarray.DataArray 'tas' (ensemble_member: 1, time: 360,
                         projection_y_coordinate: 112,
                         projection_x_coordinate: 82)>
array([[[[12.960328 , 12.909746 , 12.8551235, ...,  6.873084 ,
           6.625592 ,  6.480878 ],
         [12.844946 , 12.796289 , 12.746939 , ...,  6.4200444,
           6.290046 ,  6.1848907],
         [12.728203 , 12.682791 , 12.637478 , ...,  5.9118495,
           5.8598638,  5.7775555],
         ...,
         [ 6.377298 ,  6.390607 ,  6.3984194, ...,  5.6672983,
           5.1292915,  4.122913 ],
         [ 6.4750466,  6

2024-07-18 10:22:22,001 - distributed.worker - WARNING - Compute Failed
Key:       _calculate_single_year_indicators-ce1de2d3a84a3655b81dc914ce7f3f27
Function:  _calculate_single_year_indicators
args:      (<hazard.sources.ukcp18_rcp85.Ukcp18Rcp85 object at 0x179c5aa90>, BatchItem(resource=HazardResource(hazard_type='ChronicHeat', group_id='', path='chronic_heat/osc/v2/days_tas_above_{temp_c}c_{gcm}_{scenario}_{year}', indicator_id='days_tas/above/{temp_c}c', indicator_model_id=None, indicator_model_gcm='{gcm}', params={'temp_c': ['15'], 'gcm': ['ukcp18']}, display_name='Days with average temperature above {temp_c}°C/{gcm}', display_groups=['Days with average temperature above'], description="Days per year for which the average near-surface temperature 'tas' is above a threshold specified in °C.\n\n$$\nI =  \\frac{365}{n_y} \\sum_{i = 1}^{n_y} \\boldsymbol{\\mathbb{1}}_{\\; \\, T^{avg}_i > T^\\text{ref}} \\nonumber\n$$\n\n$I$ is the indicator, $T^\\text{avg}_i$ is the daily average nea

[2024-07-18 10:22:22,002] {indicator_model.py:37} ERROR - Batch item failed
Traceback (most recent call last):
  File "/Users/ciaran/dev/os-climate-hazard/src/hazard/indicator_model.py", line 35, in run_all
    self.run_single(item, source, target, client)
  File "/Users/ciaran/dev/os-climate-hazard/src/hazard/models/multi_year_average.py", line 78, in run_single
    averaged_indicators = self._averaged_indicators(client, source, target, item)
  File "/Users/ciaran/dev/os-climate-hazard/src/hazard/models/multi_year_average.py", line 117, in _averaged_indicators
    single_year_sets: List[List[Indicator]] = list(client.gather(futures))  # indicators for each year
  File "/Users/ciaran/dev/os-climate-hazard/.venv/lib/python3.9/site-packages/distributed/client.py", line 2361, in gather
    return self.sync(
  File "/Users/ciaran/dev/os-climate-hazard/.venv/lib/python3.9/site-packages/distributed/utils.py", line 351, in sync
    return sync(
  File "/Users/ciaran/dev/os-climate-hazard/.ven

### Plot one of the indicators 

In [ ]:
from hazard.sources.osc_zarr import OscZarr  # noqa: E402
import os, os.path  # noqa: E402

test_output_dir = os.path.join(os.getcwd(), "src/test/test_output")
store = zarr.DirectoryStore(os.path.join(test_output_dir, 'hazard', 'hazard.zarr'))
target = OscZarr(store=store)
da = target.read("chronic_heat/osc/v2/days_tas_above_15c_NorESM2-MM_ssp585_2030")
da.plot()

### List contents of 'directory'

Use s3fs to treat like directory structure, but some care needed:

- s3fs caches so care needed if objects are changed outside of s3fs
- Best to use boto3 client for handling large number of objects (see below)

In [ ]:
import s3fs  # noqa: E402

s3 = s3fs.S3FileSystem(
    anon=False,
    key=os.environ["OSC_S3_ACCESS_KEY"],
    secret=os.environ["OSC_S3_SECRET_KEY"],
)
s3.ls(os.environ["OSC_S3_BUCKET"] + "/hazard_test/hazard.zarr")

### Copy from one bucket to another bucket

Use boto3 client and paginator for dealing with large number of objects.

Some care needed if trying to move objects between buckets with different credentials: \
https://stackoverflow.com/questions/65577223/aws-s3-copy-object-from-one-bucket-to-another-with-different-credentials

In [ ]:
import boto3  # noqa: E402

bucket_name = os.environ["OSC_S3_BUCKET"]
prefix = "hazard_test/hazard.zarr/chronic_heat/osc/v2"

s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.environ["OSC_S3_ACCESS_KEY"],
    aws_secret_access_key=os.environ["OSC_S3_SECRET_KEY"],
)

paginator = s3_client.get_paginator("list_objects_v2")

pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

# get the list of keys with the given prefix
keys = []
for page in pages:
    for objs in page["Contents"]:
        if isinstance(objs, list):
            keys.extend([obj["Key"] for obj in objs])
        else:
            keys.append(objs["Key"])

print(keys)

In [ ]:
# Use this script to promote from hazard_test to hazard

target_bucket_name = "..."
# might be same as bucket_name

for key in keys:
    copy_source = {"Bucket": bucket_name, "Key": key}
    target_key = key.replace("hazard_test/hazard.zarr", "hazard/hazard.zarr")
    print(f"{key} to {target_key} for bucket {bucket_name}")
    s3_client.copy_object(CopySource=copy_source, Bucket=target_bucket_name, Key=target_key)